In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [45]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from imblearn.ensemble import EasyEnsembleClassifier



# Read the CSV and Perform Basic Data Cleaning

In [4]:
# https://help.lendingclub.com/hc/en-us/articles/215488038-What-do-the-different-Note-statuses-mean-

columns = [
    "loan_amnt", "int_rate", "installment", "home_ownership",
    "annual_inc", "verification_status", "issue_d", "loan_status",
    "pymnt_plan", "dti", "delinq_2yrs", "inq_last_6mths",
    "open_acc", "pub_rec", "revol_bal", "total_acc",
    "initial_list_status", "out_prncp", "out_prncp_inv", "total_pymnt",
    "total_pymnt_inv", "total_rec_prncp", "total_rec_int", "total_rec_late_fee",
    "recoveries", "collection_recovery_fee", "last_pymnt_amnt", "next_pymnt_d",
    "collections_12_mths_ex_med", "policy_code", "application_type", "acc_now_delinq",
    "tot_coll_amt", "tot_cur_bal", "open_acc_6m", "open_act_il",
    "open_il_12m", "open_il_24m", "mths_since_rcnt_il", "total_bal_il",
    "il_util", "open_rv_12m", "open_rv_24m", "max_bal_bc",
    "all_util", "total_rev_hi_lim", "inq_fi", "total_cu_tl",
    "inq_last_12m", "acc_open_past_24mths", "avg_cur_bal", "bc_open_to_buy",
    "bc_util", "chargeoff_within_12_mths", "delinq_amnt", "mo_sin_old_il_acct",
    "mo_sin_old_rev_tl_op", "mo_sin_rcnt_rev_tl_op", "mo_sin_rcnt_tl", "mort_acc",
    "mths_since_recent_bc", "mths_since_recent_inq", "num_accts_ever_120_pd", "num_actv_bc_tl",
    "num_actv_rev_tl", "num_bc_sats", "num_bc_tl", "num_il_tl",
    "num_op_rev_tl", "num_rev_accts", "num_rev_tl_bal_gt_0",
    "num_sats", "num_tl_120dpd_2m", "num_tl_30dpd", "num_tl_90g_dpd_24m",
    "num_tl_op_past_12m", "pct_tl_nvr_dlq", "percent_bc_gt_75", "pub_rec_bankruptcies",
    "tax_liens", "tot_hi_cred_lim", "total_bal_ex_mort", "total_bc_limit",
    "total_il_high_credit_limit", "hardship_flag", "debt_settlement_flag"
]

target = ["loan_status"]

In [29]:
# Load the data
file_path = Path('../Resources/LoanStats_2019Q1.csv.zip')
df = pd.read_csv(file_path, skiprows=1)[:-2]
df = df.loc[:, columns].copy()

# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')

# Drop the null rows
df = df.dropna()

# Remove the `Issued` loan status
issued_mask = df['loan_status'] != 'Issued'
df = df.loc[issued_mask]

# convert interest rate to numerical
df['int_rate'] = df['int_rate'].str.replace('%', '')
df['int_rate'] = df['int_rate'].astype('float') / 100


# Convert the target column values to low_risk and high_risk based on their values
x = {'Current': 'low_risk'}   
df = df.replace(x)

x = dict.fromkeys(['Late (31-120 days)', 'Late (16-30 days)', 'Default', 'In Grace Period'], 'high_risk')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()


le = LabelEncoder()

df_1 = df.copy()
df_1 = pd.get_dummies(df, columns=["home_ownership", "verification_status","issue_d","initial_list_status","next_pymnt_d","application_type"])

le.fit(df_1["pymnt_plan"])
df_1["pymnt_plan"] = le.transform(df_1["pymnt_plan"])

le.fit(df_1['hardship_flag'])
df_1['hardship_flag'] = le.transform(df_1['hardship_flag'])

le.fit(df_1['debt_settlement_flag'])
df_1['debt_settlement_flag'] = le.transform(df_1['debt_settlement_flag'])

# Split the Data into Training and Testing

In [30]:
# Create our features
X = df_1.drop(columns='loan_status')

# Create our target
y = df_1['loan_status']

In [31]:
X.describe()
y = pd.DataFrame(df_1['loan_status'])
y

,loan_status
0,low_risk
1,low_risk
2,low_risk
3,low_risk
4,low_risk
...,...
68812,low_risk
68813,low_risk
68814,low_risk
68815,low_risk


In [32]:
# Check the balance of our target values
y['loan_status'].value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [33]:
# Split the X and y into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X_train

,loan_amnt,int_rate,installment,annual_inc,pymnt_plan,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,verification_status_Verified,issue_d_Feb-2019,issue_d_Jan-2019,issue_d_Mar-2019,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App
3903,1600.0,0.0881,50.74,35964.0,0,24.12,0.0,0.0,20.0,0.0,...,0,0,0,1,1,0,0,1,1,0
28390,9000.0,0.0756,280.21,41000.0,0,16.89,1.0,0.0,16.0,0.0,...,0,1,0,0,0,1,0,1,1,0
15470,10000.0,0.1033,214.10,112000.0,0,17.75,0.0,0.0,14.0,0.0,...,1,1,0,0,0,1,0,1,1,0
4279,36000.0,0.1033,1167.21,120000.0,0,19.95,0.0,1.0,13.0,0.0,...,0,0,0,1,0,1,0,1,1,0
57514,18000.0,0.0881,570.81,51000.0,0,19.11,1.0,0.0,16.0,0.0,...,1,0,1,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49100,8400.0,0.0646,257.30,37992.0,0,33.16,0.0,0.0,9.0,0.0,...,0,0,1,0,0,1,1,0,1,0
20609,30000.0,0.0646,918.93,217000.0,0,19.56,0.0,2.0,25.0,0.0,...,0,1,0,0,0,1,1,0,1,0
21440,20000.0,0.1180,442.88,35000.0,0,16.19,0.0,0.0,6.0,0.0,...,0,1,0,0,0,1,1,0,0,1
50057,11625.0,0.1447,273.34,30000.0,0,18.96,0.0,2.0,10.0,0.0,...,0,0,1,0,0,1,1,0,1,0


# Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble AdaBoost classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Print the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier onely, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [34]:
# Resample the training data with the RandomOversampler
from imblearn.ensemble import BalancedRandomForestClassifier

brf_classifier = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf_classifier.fit(X_train, y_train)

BalancedRandomForestClassifier(bootstrap=True, class_weight=None,
                               criterion='gini', max_depth=None,
                               max_features='auto', max_leaf_nodes=None,
                               min_impurity_decrease=0.0, min_samples_leaf=2,
                               min_samples_split=2,
                               min_weight_fraction_leaf=0.0, n_estimators=100,
                               n_jobs=1, oob_score=False, random_state=1,
                               replacement=False, sampling_strategy='auto',
                               verbose=0, warm_start=False)

In [36]:
# Calculated the balanced accuracy score
y_pred = brf_classifier.predict(X_test)
accuracy_score = balanced_accuracy_score(y_test, y_pred)
accuracy_score

0.7554515069140216

In [37]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[   62,    39],
       [ 1761, 15343]])

In [38]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.03      0.61      0.90      0.06      0.74      0.54       101
   low_risk       1.00      0.90      0.61      0.94      0.74      0.57     17104

avg / total       0.99      0.90      0.62      0.94      0.74      0.57     17205



In [54]:
#method 1
importances = brf_classifier.feature_importances_
importances_sorted = sorted(zip(brf_classifier.feature_importances_, X.columns), reverse=True)
importances_sorted[:10]

[(0.07456535414747449, 'total_rec_prncp'),
 (0.06940765607552381, 'last_pymnt_amnt'),
 (0.06711648870369957, 'total_pymnt_inv'),
 (0.06297922982880659, 'total_pymnt'),
 (0.04655670310399982, 'total_rec_int'),
 (0.03509242669167666, 'int_rate'),
 (0.022467652445406414, 'issue_d_Jan-2019'),
 (0.01844072790833401, 'bc_util'),
 (0.018299748883261302, 'out_prncp_inv'),
 (0.01725825076565536, 'dti')]

In [44]:
#method 2
pd.set_option('display.max_rows', 100)
feature_importances = pd.DataFrame(brf_classifier.feature_importances_, index = X_train.columns, columns=['importance']).sort_values('importance', ascending=False)
feature_importances

,importance
total_rec_prncp,0.074565
last_pymnt_amnt,0.069408
total_pymnt_inv,0.067116
total_pymnt,0.062979
total_rec_int,0.046557
int_rate,0.035092
issue_d_Jan-2019,0.022468
bc_util,0.018441
out_prncp_inv,0.018300
dti,0.017258


### Easy Ensemble AdaBoost Classifier

In [46]:
# Train the Classifier
ee_classifier = EasyEnsembleClassifier(n_estimators=100,random_state=1)
ee_classifier.fit(X_train, y_train)

EasyEnsembleClassifier(base_estimator=None, n_estimators=100, n_jobs=1,
                       random_state=1, replacement=False,
                       sampling_strategy='auto', verbose=0, warm_start=False)

In [47]:
# Calculated the balanced accuracy score
y_pred_2 = ee_classifier.predict(X_test)
balanced_accuracy_score(y_test, y_pred_2)

0.9328878543841288

In [48]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_2)

array([[   93,     8],
       [  941, 16163]])

In [49]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_2))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.09      0.92      0.94      0.16      0.93      0.87       101
   low_risk       1.00      0.94      0.92      0.97      0.93      0.87     17104

avg / total       0.99      0.94      0.92      0.97      0.93      0.87     17205

